In [1]:
%pip install pandas seaborn pip install memory-profiler --quiet

Note: you may need to restart the kernel to use updated packages.


In [17]:
import pyarrow.parquet as pq
import pyarrow as pa
from pyarrow import csv
import pandas as pd
import re
import json

# read in parquet file for sprucehall data
dataset = pq.read_table('sprucehall')

sprucehallFolder = dataset.to_pandas()

# uniqueSpruce holds all unique point labels in data
uniqueSpruce = sprucehallFolder.groupby('point').size().reset_index(name='counts')

In [18]:
def tokenize(row, buildingdict, number):
    dicttoken = {}
    currindex = 0
    
    # key of each entry into point label dictionary will be the index of the beginning of each value
    
    while currindex < len(row):
        
        # Delimiters
        if row[currindex] in {"$", ":", "_"}:
            dicttoken[currindex] = row[currindex]
            currindex += 1
            continue
        
        # Continuous upper
        if row[currindex].isupper() and (currindex + 1 < len(row) and row[currindex + 1].isupper()):
            concatenateToken = ""
            startindex = currindex
            while currindex < len(row) and row[currindex].isupper():
                concatenateToken += row[currindex]
                currindex += 1
            dicttoken[startindex] = concatenateToken
            continue
        
        # Lower following upper
        if row[currindex].isupper() and (currindex + 1 < len(row) and row[currindex + 1].islower()):
            concatenateToken = ""
            startindex = currindex
            concatenateToken += row[currindex]
            currindex += 1
            while currindex < len(row) and row[currindex].islower():
                concatenateToken += row[currindex]
                currindex += 1
            dicttoken[startindex] = concatenateToken
            continue
        
        # Repeating digits
        if row[currindex].isdigit():
            concatenateToken = ""
            startindex = currindex
            while currindex < len(row) and row[currindex].isdigit():
                concatenateToken += row[currindex]
                currindex += 1
            dicttoken[startindex] = concatenateToken
            continue
        
        # Increment the current index if no conditions are met
        currindex += 1
        
    buildingdict[number] = dicttoken

In [19]:
# building labels is the main dictionary
# number will be used as the key for each point label's dictionary
buildinglabels = {}
number = 0

# for loop looping through each row
for row, count in uniqueSpruce.iterrows():  
    point_label = uniqueSpruce.iloc[row, 0]
    tokenize(point_label, buildinglabels, number)
    number += 1

# making json file with main directory
jsonlabels = json.dumps(buildinglabels, indent = 4)

# creation of json file
with open("pointlabels.json", "w") as outfile:
    outfile.write(jsonlabels)